In [ ]:
from skimage.metrics import structural_similarity
import cv2
import numpy as np
import math

#LOADING IMAGES
before_big = cv2.imread("C:/Users/WOOBW65/Documents/bap/cropper/datasetdartss2/cropped/20200302_184545.jpg")
after_big = cv2.imread("C:/Users/WOOBW65/Documents/bap/cropper/datasetdartss2/cropped/20200302_184552.jpg")

#deze images kunnen soms te groot zijn en dan kijkt het programma te gedetaileerd, een van de manieren om dit te vermijden is dus scalen
scale_percent = 20
width = int(before_big.shape[1] * scale_percent / 100)
height = int(after_big.shape[0] * scale_percent / 100)
dim = (width, height)

before = cv2.resize(before_big, dim, interpolation = cv2.INTER_AREA)
after = cv2.resize(after_big, dim, interpolation = cv2. INTER_AREA)

# convert the images to grayscale
before_gray = cv2.cvtColor(before, cv2.COLOR_BGR2GRAY)
after_gray = cv2.cvtColor(after, cv2.COLOR_BGR2GRAY)

#jpg en png zorg voor redelijk wat ruis, misschien weg doen?
#denoise = cv2.fastNlMeansDenoising(before_gry,None,20,10,7,21)
#denoise = cv2.fastNlMeansDenoising(after_gry,None,20,10,7,21)

# verschil tussen de 2 afbeeldingen berekenen, de score geeft de % gelijkheid tussen de images weer
(score, diff) = structural_similarity(before_gray, after_gray, full=True)
print("Image similarity", score)

# The diff image contains the actual image differences between the two images
# and is represented as a floating point data type in the range [0,1] 
# so we must convert the array to 8-bit unsigned integers in the range
# [0,255] before we can use it with OpenCV
diff = (diff * 255).astype("uint8")

# Threshold the difference image, followed by finding contours to
# obtain the regions of the two input images that differ
thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

#mask
mask = np.zeros(before.shape, dtype='uint8')
filled_after = after.copy()

#Alle countours
for c in contours:
    #bounding rectangle rondzetten
    area = cv2.contourArea(c)
    if area > 40:
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(mask, (x, y), (x + w, y + h), (36,255,12), 2)
        cv2.rectangle(after, (x, y), (x + w, y + h), (36,255,12), 2)
        cv2.drawContours(mask, [c], 0, (0,255,0), -1)
        cv2.drawContours(filled_after, [c], 0, (0,255,0), -1)
        
        #normaal zit een pijtlje verticaal, indien toevallig horizontaal houden we hier rekening mee
        omgekeerd = False        
        if(h < w):
            omgekeerd = True
        
        #https://prnt.sc/raenhl dit zijn de coordinaten
        #wanneer een pijl horizontaal zit zullen we dus kijken of de rechterkant of linkerkant meer pixels heeft
        #de staart is dik en zal altijd meer pixels hebben
        #zo kunnen we bepalen in welk vak de punt zit
        if(omgekeerd):
            #tekent de lijn die de mask juist opsplitst
            cv2.line(mask, (int((2 * x + w) / 2), y), (int((2 * x + w) / 2), y + h), (255 , 0, 0), 3)
            totalRight = 0
            totalLeft = 0
            #alle punten overlopen en elke pixel bijhouden in welk stuk het zit
            for p in c:
                if(p[0][0] > ((2 * x + w) / 2)):
                    totalRight += 1
                else:
                    totalLeft += 1
            #rechterkant heeft meer pixels
            if(totalRight > totalLeft):
                #alle punten in het speciefiek vak overlopen en daar 1 pixel uithalen die die punt van de pijl is
                for p in c:
                    last = 500
                    #de punt zit links dus x zo klein mogelijk
                    if(p[0][0] < last):
                        point = p
            else:
                for p in c:
                    last = 0
                    #punt zit rechts dus x zo groot mogelijk
                    if(p[0][0] > last):
                        point = p
            
            #cirkel tekenen op de punten
            cv2.circle(before, (p[0][0], p[0][1]), 3, (255, 0, 0), -15)
            cv2.circle(mask, (p[0][0], p[0][1]), 3, (255, 0, 0), -15)
            cv2.circle(after, (p[0][0], p[0][1]), 3, (255, 0, 0), -15)

        else:
            cv2.line(mask, (int(x), int((y + h + y) /2)), (int((x + w)), int((y + h + y) /2)), (255 , 0, 0), 3)
            totalUp = 0
            totalDown = 0
            for p in c:
                #som maken met xjes zal hier nauwkeuriger zijn // je kan gewoon een teller maken
                if(p[0][1] > ((2 * y + h) / 2)):
                    totalDown += 1
                else:
                    totalUp += 1
            
            if(totalDown > totalUp):
                for p in c:
                    last = 0
                    if(p[0][1] > last):
                        point = p
            else:
                for p in c:
                    last = 500
                    if(p[0][1] < last):
                        point = p
            
            cv2.circle(before, (p[0][0], p[0][1]), 3, (0, 0, 255), -15)
            cv2.circle(mask, (p[0][0], p[0][1]), 3, (0, 0, 255), -15)
            cv2.circle(after, (p[0][0], p[0][1]), 3, (0, 0, 255), -15)
            


def calculateScore(dart):
    print(dart)
    #crop programma knipt het bord uit met het centrum exact in het midden en afmetingen 1520 * 1520
    #750 is dus het midden van de foto en bord, we houden rekening met de scale
    middle = int(750 * scale_percent / 100)
    
    #afstand van punt tot middelpunt, hiermee kunnen we bepalen of het in trippel, dubbel of enkel zit
    distance = math.sqrt(pow(dart[0][0] - (750 * scale_percent / 100), 2) + pow(dart[0][1] - (750 * scale_percent / 100), 2))
    
    #hoek berekenen, hiermee kunnen we zien in welk vakje hij zit
    angle = (math.degrees(math.atan2(150-dart[0][1],dart[0][0] - 150))+360)%360

    #score aan hoek hangen
    finalscore = 0
    if 0 < angle < 9:
        finalscore = 6
    elif 0 < angle < 27:
        finalscore = 13
    elif 27 < angle < 45:
        finalscore = 4
    elif 45 < angle < 63:
        finalscore = 18
    elif 63 < angle < 81:
        finalscore = 1
    elif 81 < angle < 99:
        finalscore = 20
    elif 99 < angle < 117:
        finalscore = 5
    elif 117 < angle < 135:
        finalscore = 12
    elif 135 < angle < 153:
        finalscore = 9
    elif 153 < angle < 171:
        finalscore = 14
    elif 171 < angle < 189:
        finalscore = 11
    elif 189 < angle < 207:
        finalscore = 8
    elif 189 < angle < 225:
        finalscore = 16
    elif 225 < angle < 243:
        finalscore = 7
    elif 243 < angle < 261:
        finalscore = 19
    elif 261 < angle < 279:
        finalscore = 3
    elif 279 < angle < 297:
        finalscore = 17
    elif 297 < angle < 315:
        finalscore = 2
    elif 315 < angle < 333:
        finalscore = 15
    elif 333 < angle < 351:
        finalscore = 10
    else:
        finalscore = 6
        
    #kijken of hij in dubbel, trippel, roos, buitenroos of naast het bord zit
    if 330 * scale_percent / 100 < distance < 365 * scale_percent / 100:
        finalscore * 3
    elif 500 * scale_percent / 100 < distance < 590 * scale_percent / 100:
        finalscore * 2
    elif distance > 590 * scale_percent / 100:
        finalscore = 0
    elif distance < 20 * scale_percent / 100:
        finalscore = 50
    elif 20 * scale_percent / 100 < distance < 50 * scale_percent / 100:
        finalscore = 25
    
    
    print("finalscore  = ")
    print(finalscore)
            
    
calculateScore(point)
           

cv2.imshow('before', before)
cv2.imshow('after', after)
cv2.imshow('diff', diff)
cv2.imshow('mask', mask)
cv2.imshow('filled after', filled_after)
cv2.waitKey(0)

Image similarity 0.9507553687165303
[[ 68 167]]
finalscore  = 
8
